In [ ]:
from nornir import InitNornir
import pynetbox
from jinja2 import Environment, FileSystemLoader
import json
from pprint import pprint
from deepdiff import DeepDiff
from deepdiff import Delta
from deepdiff.serialization import json_dumps, json_loads
import re

In [ ]:
# Init Nornir
nr = InitNornir(
    inventory={
        "plugin": "NetBoxInventory2",
        "options": {
            "nb_url": "http://10.30.1.226:8000",
            "nb_token": "dc5b766afda60e647a9fcc384c736a9689da8618",
            "group_file": "./inventory/groups.yml",
            "defaults_file": "./inventory/defaults.yml",
        },
    },
)

In [ ]:
# Init pynetbox
nb_url = nr.config.inventory.options['nb_url']
nb_token = nr.config.inventory.options['nb_token']
nb = pynetbox.api(
    nb_url,
    token=nb_token
)

In [ ]:
templates_path = "./templates/"

In [ ]:
changes2 = list([{
    'id': 136, 'mtu': 1500, 'mac_address': None, 'speed': 1000000, 'duplex': 'full', 'description': 'T-SIDOROV', 'mode': 'access', 'untagged_vlan': 8 
}])

In [ ]:
""" template_file = "netbox_interface_id.template"
environment = Environment(loader=FileSystemLoader(templates_path)) # загружаем шаблон для заполнения
template = environment.get_template(template_file)
content = None """

device_intf_id1 = 136
device_intf_id2 = 606
interface = ['mtu','mac_address','speed','duplex','description','mode','untagged_vlan']
get_device_interface1 = nb.dcim.interfaces.get(device_intf_id1) # по ID находим интерфейс в netbox
get_device_interface2 = nb.dcim.interfaces.get(device_intf_id2)
dev1 = list(get_device_interface1)
dev2 = list(get_device_interface2)
data = []
changes = dict()
change_key = ['id']
new_value = [device_intf_id1]

for _ in range(len(dev2)):

    if dev2[_][0] in interface and dev2[_][1] != '' and dev2[_][1] != None:
    #if dev2[_][0] in interface:
        change_key.append(dev2[_][0])
        if isinstance(dev2[_][1], dict):
            new_value.append(list(dev2[_][1].values())[0])
        else:
            new_value.append(dev2[_][1])
changes = [dict(zip(change_key,new_value))]

print(changes)
#nb.dcim.interfaces.update(changes)